# LangChain with Simple Document As Context to LLM

In [ ]:
import sys
sys.path.insert(1, '../../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
# print(AZURE_OPENAI_API_KEY)
# print(AZURE_OPENAI_ENDPOINT)
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

In [ ]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"
os.environ["AZURE_OPENAI_API_VERSION"]="2024-07-01-preview"


from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

## LLM

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Help user answer questions"),
    HumanMessage(content="Who is Noushath?"),
]

reply=model.invoke(messages)
print(reply.content)

## Call LLM with custom Document as input along with query

## Load Data

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load the reference document
loader=PyPDFLoader("../materials/NoushathDPNProfile.pdf")
data=loader.load()
print(data)

## Transform Data

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=20,
                                      length_function=len,
                                      separator=".")
final_data= text_splitter.split_documents(data)
print(final_data)

In [ ]:
len(final_data)

## Embedding Model

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

## Vector Store

In [ ]:
from langchain.vectorstores import FAISS # FAISS is a vector store

faissdb=FAISS.from_documents(final_data,embeddings)


## Let's see embeddings

In [ ]:
faissdb.docstore.__dict__

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(model,chain_type="stuff")


### Run this cell to see behavior with hugging face model locally

In [ ]:
# try with hugging face

# from langchain.llms import HuggingFacePipeline
# llmHfcp = HuggingFacePipeline.from_model_id(
#     model_id="bigscience/bloom-1b7",
#     task="text-generation",
#     model_kwargs={"temperature": 0.7, "max_length": 200,"do_sample":'True'},
# )
# print(llmHfcp("Who is Noushath Moidu?"))
# chain=load_qa_chain(HuggingFacePipeline.from_model_id(
#     model_id="bigscience/bloom-1b7",
#     task="text-generation",
#     model_kwargs={"temperature": 0.7, "max_length": 200,"do_sample":'True'},
# ),chain_type="stuff")

In [ ]:
query="Who is Noushath Moidu?"
# Facebook AI Similarity Search (Faiss), a library that allows us to quickly search for multimedia documents
docsFoundFromFAISS=faissdb.similarity_search(query)

print(docsFoundFromFAISS)

## Retriever

In [ ]:
query="Who is Noushath Moidu?"
retriever = faissdb.as_retriever(search_kwargs={"k":3})
docsFoundFromFAISS =retriever.get_relevant_documents(query)
print(docsFoundFromFAISS)

In [ ]:
chain.run(input_documents=docsFoundFromFAISS, question=query)

In [ ]:
from langchain.chains import RetrievalQA
ragChain=RetrievalQA.from_llm(llm=model,retriever=faissdb.as_retriever(search_kwargs={"k":1}), return_source_documents=True)
ragChain.invoke(input=query)

In [ ]:
chain.run(input_documents=docsFoundFromFAISS, question="How long have been Noushath Moidu working?")

In [ ]:
chain.run(input_documents=docsFoundFromFAISS, question="Please tell me how long have been Noushath Moidu working and how did you reach this answer")

In [ ]:
chain.run(input_documents=docsFoundFromFAISS, question="Please tell me how long have been Noushath Moidu working also explain me how you reached this answer")

In [ ]:
chain.run(input_documents=docsFoundFromFAISS, question="Please tell me how long have been Noushath Moidu working also explain me how you reached this answer and also tell me the context")